# Main NoteBook - Random Walk + AprilTag Detection + Action + Avoid Obstacle

In [1]:
# Imports
import time
import sys
import math
from random import randint
import asyncio
from IPython.display import clear_output
import cv2                                # state of the art computer vision algorithms library
import numpy as np                        # fundamental package for scientific computing
import matplotlib.pyplot as plt           # 2D plotting library producing publication quality figures
import pyrealsense2 as rs                 # Intel RealSense cross-platform open-source API
from pyapriltags import Detector

from unitree_sdk2py.core.channel import ChannelSubscriber, ChannelFactoryInitialize
from unitree_sdk2py.idl.default import unitree_go_msg_dds__SportModeState_
from unitree_sdk2py.idl.unitree_go.msg.dds_ import SportModeState_
from unitree_sdk2py.go2.video.video_client import VideoClient
from unitree_sdk2py.go2.sport.sport_client import (
    SportClient,
    PathPoint,
    SPORT_PATH_POINT_SIZE,
)
from unitree_sdk2py.go2.obstacles_avoid.obstacles_avoid_client import ObstaclesAvoidClient
from unitree_sdk2py.go2.vui.vui_client import VuiClient

%matplotlib inline

In [2]:
# setup for add on camera
ChannelFactoryInitialize(0, "eth0")
# ChannelFactoryInitialize()

sport_client = SportClient()  
sport_client.SetTimeout(10)
sport_client.Init()

obsclient = ObstaclesAvoidClient()
obsclient.SetTimeout(3.0)
obsclient.Init()

# setup for default camera
client = VideoClient()
client.SetTimeout(1.0)
client.Init()

vui_client = VuiClient()
vui_client.SetTimeout(3.0)
vui_client.Init()


In [3]:
at_detector = Detector(families='tag36h11',
                       nthreads=1,
                       quad_decimate=1.0,
                       quad_sigma=0.5,
                       refine_edges=1,
                       decode_sharpening=0.25,
                       debug=0)

In [4]:
def blink_disco_mode():
    pattern = [0, 3, 6, 9, 11, 7, 4, 2]  # A rhythmic pattern
    for i in range(8):
        for level in pattern:
            vui_client.SetBrightness(level)
            time.sleep(0.1)  # Adjust for desired speed
    vui_client.SetBrightness(0)

def blink_wiggle_hips():
    pattern = [0, 3, 6, 9, 11, 7, 4, 2]  # A rhythmic pattern
    for i in range(3):
        for level in pattern:
            vui_client.SetBrightness(level)
            time.sleep(0.1)  # Adjust for desired speed
    vui_client.SetBrightness(0)

def blink_starting_stopping():
    for i in range(6):
        level = 5 if i%2 else 0
        time.sleep(0.1)
    vui_client.SetBrightness(0)

def blink_standup():
    for i in range(12):
        vui_client.SetBrightness(i)
        time.sleep(0.1)
    vui_client.SetBrightness(0)

def blink_sitdown():
    for i in reversed(range(12)):
        vui_client.SetBrightness(i)
        time.sleep(0.1)
    vui_client.SetBrightness(0)

def blink_hello():
    for i in range(2):
        level = 4 if i%2 else 0
        vui_client.SetBrightness(level)
        time.sleep(0.5)
    vui_client.SetBrightness(0)

def blink_stretch():
    pattern = [2, 5, 8, 4, 1, 3]  # Mimicking a stretching movement
    sleep_times = [0.1, 0.6, 0.2, 0.5, 0.3, 0.6]  # Slow and relaxing pattern
    for i, level in enumerate(pattern):
        vui_client.SetBrightness(level)
        time.sleep(sleep_times[i])
    vui_client.SetBrightness(0)


In [5]:
def standup():
    sport_client.StandUp()
    blink_standup()
    sport_client.BalanceStand()
    time.sleep(1)
    
def move_randomly(vx=0.3, vy=0.3, vyaw=0.3):
    obsclient.UseRemoteCommandFromApi(True)
    obsclient.Move(abs(vx), vy, vyaw)
    # time.sleep(2)

def move_back():
    obsclient.UseRemoteCommandFromApi(True)
    obsclient.Move(-0.2, 0, 0)
    time.sleep(1.5)

def stop_move():
    obsclient.Move(0.0, 0, 0)
    time.sleep(1)
    sport_client.BalanceStand()
    time.sleep(1)
    
def sitdown():
    obsclient.Move(0.0, 0, 0)
    time.sleep(1)
    sport_client.StandDown()
    blink_sitdown()

def act(code):
    stop_move()

    if code == 0:
        print("hello")
        sport_client.Hello()
        blink_hello()
    elif code == 1:
        print("Heart")
        sport_client.Heart()
    elif code == 2:
        print("Stretch")
        sport_client.Stretch()
        blink_stretch()
        sport_client.BalanceStand()
    elif code == 3:
        print("WiggleHips")
        sport_client.WiggleHips()
        blink_wiggle_hips()
    elif code == 4:
        print("SitDown + StandUp")
        sport_client.StandDown()
        blink_sitdown()
        time.sleep(2)
        sport_client.StandUp()
        blink_standup()
        time.sleep(2)
    elif code == 5:
        print("Dance 1")
        sport_client.Dance1()
        blink_disco_mode()
    else:
        return
   
    #end
    sport_client.BalanceStand()
    time.sleep(1)
    move_randomly()
    # time.sleep(3)
  

In [6]:
# camera Setup:
pipe = rs.pipeline()
cfg = rs.config()
profile = pipe.start(cfg)

In [7]:
def detect_tag_from_main_camera():
    try:
        # feed from default camera
        code,data = client.GetImageSample()
        img_1 = np.frombuffer(bytes(data), dtype=np.uint8)
        image_1 = cv2.imdecode(img_1, cv2.IMREAD_COLOR)
        image_1_gray = cv2.cvtColor(image_1, cv2.COLOR_BGR2GRAY)
        tags = at_detector.detect(image_1_gray)
        tags_10 =  [t for t in tags if t.tag_id == 10]
        return tags_10
    except Exception as ex:
        return []
    
    
def detect_tag():
    ignore = [10]
    frameset = pipe.wait_for_frames()
    color_frame = frameset.get_color_frame()
    color_rgb = np.asanyarray(color_frame.get_data())
    center_crop = color_rgb[:,240:-240,:]
    center_crop_gray = cv2.cvtColor(center_crop, cv2.COLOR_RGB2GRAY)
    tags = at_detector.detect(center_crop_gray)
    return [t.tag_id for t in tags if t.tag_id not in ignore]

# Using AprilTag avoid the boundry

In [8]:
sitdown()
threshold = 950
move = False

while True:
    if move:
        tags_2 = detect_tag_from_main_camera()  
        found = False
        for tag in tags_2:
            if tag.center[1] < threshold:
                found = True
                print("found tag inside the box")
                break
                
        if len(tags_2)> 0 and not found:
            print(f"Detected {len(tags_2)} but no tags in focus area")
           
        if not found:
            # Move back for a sec
            print("No tags detected ... moving back")
            move_back()
        
    tags = detect_tag()

    if 12 in tags:
        print("stopping .. sitting down")
        stop_move()
        blink_starting_stopping()
        sitdown()
        move = False
    elif 15 in tags:
        blink_starting_stopping()
        if not move:
            standup()
        print("Starting")
        move = True
    elif len(tags) > 0:
        act(tags[0])
        
    if move:
        # r = (np.random.random(3)-0.5)*0.6
        move_randomly()

found tag inside the box
found tag inside the box
found tag inside the box
No tags detected ... moving back
found tag inside the box
found tag inside the box
found tag inside the box
found tag inside the box
No tags detected ... moving back
found tag inside the box
found tag inside the box
found tag inside the box
WiggleHips
found tag inside the box
found tag inside the box
found tag inside the box
found tag inside the box
No tags detected ... moving back
found tag inside the box
found tag inside the box
No tags detected ... moving back
found tag inside the box
found tag inside the box
No tags detected ... moving back
found tag inside the box
found tag inside the box
found tag inside the box
found tag inside the box
found tag inside the box
No tags detected ... moving back
found tag inside the box
found tag inside the box
found tag inside the box
found tag inside the box
found tag inside the box
found tag inside the box
found tag inside the box
found tag inside the box
found tag inside

KeyboardInterrupt: 

In [9]:
stop_move()